In [3]:
import os
import pickle
import pandas as pd
import numpy as np
import networkx as nx


In [56]:
%%time

## Creating the mapping
## 1. transactionHash- Integer ID mapping

list_tx_hashes = []
path = r"C:\Users\japje\BDAProj\edges2010"
for root, dir, files in os.walk(path):
    for f in files:
        with open(os.path.join(root,f), encoding = "utf-8") as f:
            for line in f:
                ## Split the row
                cells = line.split("\t")

                ## Add transaciton to Set
                transaction_hash = cells[1]
                list_tx_hashes.append(transaction_hash)
                
                
list_tx_hashes = list(set(list_tx_hashes))
txn_mapping = dict(zip(list_tx_hashes, range(len(list_tx_hashes))))
del list_tx_hashes

Wall time: 308 ms


In [9]:
%%time

## Creating the mapping
##  Address- Integer ID mapping

list_addresses = []
path = r"C:\Users\japje\BDAProj\edges2010"
for root, dir, files in os.walk(path):
    for f in files:
        if("outputs" in f):
            with open(os.path.join(root,f), encoding = "utf-8") as f:
                for line in f:
                    ## Split the row
                    cells = line.split("\t")

                    ## Add address to Set
                    addresses = cells[2:][::2]
                    list_addresses = list_addresses + addresses
                
                
list_addresses = list(set(list_addresses))
addr_mapping = dict(zip(list_addresses, range(len(list_addresses))))
del list_addresses

Wall time: 2min 26s


In [94]:
##Serialize Txn and Addr mappings

# import pickle
# path_txn_mapping2010 = r"C:\Users\japje\BDAProj\Working2010\txnHashtoID_mapping_2010.pickle"
# path_addr_mapping2010 = r"C:\Users\japje\BDAProj\Working2010\AddrToID_mapping_2010.pickle"
# with open(path_txn_mapping2010, "wb") as f:
#     pickle.dump(txn_mapping, f)

# with open(path_addr_mapping2010, "wb") as f1:
#     pickle.dump(addr_mapping, f1)
    

In [87]:
#testing parse logic

# sample_op_row = "1279128036	912c869c1ff6acaa9ca9d38ee66933c167ac980ee1c3e20086e3e6dd6a8896ef	18gBZnsuSrhYLjvPUgwvDUJmksfREUGBTT	1000000	1M5o4Es83Ls4UJ3md94FbLbebLhWzF8eQc	3079000000"

# cells = sample_op_row.split("\t")

In [116]:
%%time
## Generation of output mappings
## 1. txnID -> output_index -> OutputID
## 2. OutputID -> addressID -> amount

list_addresses = []
path = r"C:\Users\japje\BDAProj\edges2010"
mapping_tx_to_op = {}
mapping_op = {}
global_index = 0
for root, dir, files in os.walk(path):
    for f in files:
        if("outputs" in f):
            with open(os.path.join(root,f), encoding = "utf-8") as f:
                for line in f:
                    ## Split the row
                    cells = line.rstrip().split("\t")
                    # print(cells)
                    list_tx_index = list(range(0, len(cells[2:])//2))
                    list_addr_id = list(range(global_index+1, global_index+1+len(cells[2:])//2))
                    list_tx_rep = [txn_mapping[cells[1]]] * (len(cells[2:])//2)

                    list_addresses = [addr_mapping[x] for x in cells[2:][::2]]
                    list_amounts = cells[3:][::2]

                    mapping_delta1 = {x: {y: z} for x, y, z in zip(list_tx_rep, list_tx_index, list_addr_id)}

                    mapping_delta2 = {x: {y: z} for x, y, z in zip(list_addr_id, list_addresses, list_amounts)}

                    mapping_tx_to_op.update(mapping_delta1)
                    mapping_op.update(mapping_delta2)
                    global_index = global_index+1
                global_index = global_index+1




Wall time: 1.53 s


In [118]:
len(mapping_op)

117489

In [119]:
 ## Serialize output mappings

import pickle
path_txn_to_op_mapping2010 = r"C:\Users\japje\BDAProj\Working2010\txnID_index_to_outputID_mapping_2010.pickle"
path_opID_to_addrAmt_mapping2010 = r"C:\Users\japje\BDAProj\Working2010\opID_to_addrAmt_mapping2010.pickle"

with open(path_txn_to_op_mapping2010, "wb") as f:
    pickle.dump(mapping_tx_to_op, f)

with open(path_opID_to_addrAmt_mapping2010, "wb") as f1:
    pickle.dump(mapping_op, f1)

In [15]:
## Defining 2 node classes

# class Transaction:
#     def __init__(self, id):
#         self.id = id

# class Output:
#     def __init__(self, addr_id, amount):
#         self.addr_id = addr_id
#         self.amount = amount

In [ ]:
## Reading inputs to create input mapping {transactionID --> [output indexes]}
## Adding edges

path = r"C:\Users\japje\BDAProj\edges2010"
# mapping_tx_to_op = {}
mapping_ip = {}
for root, dir, files in os.walk(path):
    for f in files:
        if("inputs" in f):
            # print(f)
            with open(os.path.join(root,f), encoding = "utf-8") as f:
                for line in f:
                    cells = line.rstrip().split("\t")
                    txn_id = txn_mapping[cells[1]]
                    
                    inputs = cells[2:]
                    input_index = []
                    for i in range(0, len(inputs)):
                        # print(str(inputs[i]) + "----"+str(inputs[i+1]))
                        try:
                            print(str(txn_mapping[inputs[i]]) + "----" + str(inputs[i+1]))
                            input_index.append(mapping_tx_to_op[int(txn_mapping[inputs[i]])][int(inputs[i+1])])
                        except:
                            pass
                        i = i+2
                    mapping_ip[txn_id] = input_index



In [159]:
 ## Serialize input mapping
import pickle
path_ip_mapping2010 = r"C:\Users\japje\BDAProj\Working2010\inp_mapping_2010.pickle"


with open(path_ip_mapping2010, "wb") as f:
    pickle.dump(mapping_ip, f)



In [ ]:
## 1. generate directed graph (NetworkX library) - Insert transactions (prefix "t-") and outputs from the output file

## Initialize
G = nx.DiGraph()

## Adding the transactions and outputs from output mappings
for k in mapping_tx_to_op:
    G.add_node("t_"+ str(k))
    connected_op = list(mapping_tx_to_op[k].values())
    # print(connected_op)
    G.add_nodes_from(connected_op)
    G.add_edges_from([("t_"+ str(k), y) for y in connected_op])
    
## Adding additional edges from input mapping
for k in mapping_ip:
    inputs = mapping_ip[k]
    # print([("t_"+ str(k), y) for y in inputs])
    G.add_edges_from([("t_"+ str(k), y) for y in inputs])


In [162]:
## Serialize Graph

nx.write_gpickle(G, r"C:\Users\japje\BDAProj\Working2010\DiGraph.pickle")

In [ ]:
# G[Transaction(12)].edges

# for n in G:
#     print(n.id)

G[t1][o3]


In [51]:

# ##testing networkx

# ## 1. generate graph

# t1 = Transaction(12)
# t2 = Transaction(123)
# t3 = Transaction(43)

# o1 = Output(12, 3400000)
# o2 = Output(15, 34000000)
# o3 = Output(126, 34000004)

# G = nx.DiGraph()
# G.add_node(t1)
# G.add_node(t2)
# G.add_node(t3)

# G.add_node(o1)
# G.add_node(o2)
# G.add_node(o3)

# G.add_edge(t1, o2)
# G.add_edge(t2, o1)
# G.add_edge(t1, o3)
# G.add_edge(t3, o1) 


In [158]:
set([len(x) for x in mapping_ip.values()])

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 76,
 79,
 80,
 84,
 92,
 96,
 101,
 105,
 106,
 117,
 118,
 119,
 131,
 136,
 162,
 175,
 199,
 205,
 279,
 287,
 314,
 363,
 384,
 387,
 451,
 490}

In [155]:
mapping_tx_to_op[96434][1]

2